In [1]:
from aiogram import Bot, Dispatcher, executor, types
from aiogram.types import ContentType
from aiogram.dispatcher.filters.state import State, StatesGroup
from aiogram.types.message import ContentTypes
from aiogram.dispatcher import FSMContext
from aiogram.contrib.fsm_storage.memory import MemoryStorage
from sqlalchemy import update

API_TOKEN = "5091011572:AAG4NfkC_zZjcsaAFkwLm4ZXOvhEqyLpQhY"


In [2]:
import asyncio

In [3]:
from food.paths import *
from food.search import *
import pandas as  pd
from food.psql import *
import pytz
timezones = pytz.all_timezones
import requests
from requests.structures import CaseInsensitiveDict
import urllib
from tzwhere import tzwhere

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
dishes = 'dishes'

In [6]:
user_id = 78882798

In [7]:
today_consumed = pd.read_sql(f"""select energy,grams,timestamp from food.{dishes} 
                                 where user_id = {user_id} and timestamp > now() - interval '24 hours'
                                 and grams is not null;""",engine).set_index("timestamp")
today_consumed= today_consumed['energy']/100*today_consumed['grams']
user_tz = engine.execute(f"""select value from food.user_properties 
                             where user_id={user_id} and
                             property='tz'
                             order by id desc limit 1""").first()

user_tz = user_tz[0] if user_tz else 'UTC'
today_consumed = today_consumed.tz_convert(user_tz)
now = pd.Timestamp.now(tz = user_tz)
today_consumed = today_consumed.reset_index()	
this_morning = pd.Timestamp(year = now.year,month = now.month,day = now.day,hour = 3,tz = user_tz)
today_consumed = today_consumed[today_consumed['timestamp'] > pd.Timestamp(this_morning)][0].sum()
today_consumed
    


433.46777777777777

In [8]:
def geocode(q):
    geocoding_key = '5d96ac126bcb462cb373297924ab2cb4'
    url = "https://api.geoapify.com/v1/geocode/search?"

    params = {"apiKey":geocoding_key, 
            "text":q}

    resp = requests.get(url + urllib.parse.urlencode(params)).json()
    return  pd.json_normalize(resp['features']).sort_values('properties.rank.importance',ascending = False)[['properties.lat','properties.lon']].iloc[0].to_list()
   

In [9]:
def get_tz(q):
    lat,lon = geocode(q)
    return tzwhere.tzwhere().tzNameAt(lat,lon)

In [10]:
async def async_get_tz(q):
    return get_tz(q)

In [11]:
async def async_search_image(url, env='prod'):
    return search_image(url,env)

In [12]:
async def async_geocode(q):
    return geocode(q)

In [13]:
async def  async_insert_on_conflict(*args, **qwargs):
    return insert_on_conflict(*args, **qwargs)


In [14]:
async def add_sender(message):
    sender = message['from'].to_python()
    sender = pd.DataFrame(sender,index=[0]).drop(columns =['is_bot'])
    await async_insert_on_conflict(sender,'users',unique_cols=['id'])

In [15]:
# Initialize bot and dispatcher
bot = Bot(token=API_TOKEN)
storage = MemoryStorage()
dp = Dispatcher(bot, storage=storage)

dishes_table = Dishes.__table__


In [16]:
ml_version = 0.2
"""0.2 - """

'0.2 - '

In [17]:
class CState(StatesGroup):
    init           = State()
    photo_taken    = State()
    measured       = State()
    set_timezone   = State()
    

In [18]:
@dp.message_handler(commands=['start'])
async def send_welcome(message: types.Message):
    
    await CState.init.set()
    await message.reply("""Counting calories as easy as taking pictures. Just capture everything before you eat it\n
                          Now send a photo of your meal to try""")

In [19]:
@dp.message_handler(commands=['set_timezone'])
async def send_welcome(message: types.Message, state: FSMContext):
    await CState.set_timezone.set()
    await message.reply(f"please search your town to set timezone")


In [20]:
@dp.message_handler(state=CState.set_timezone)
async def set_timezone(message: types.Message, state: FSMContext):
    await types.ChatActions.typing()
    await add_sender(message)
    tz = await async_get_tz(message.text)

    df = pd.DataFrame([[message['from']['id'],'tz',tz,pd.Timestamp.utcnow()]],columns = ['user_id','property','value','timestamp'])
    df.to_sql('user_properties',schema = schema,con = engine,if_exists = 'append',index = False)

    await message.reply(f"your tz is set to {tz}")

In [21]:
@dp.message_handler(content_types=ContentType.PHOTO)
async def process_photo(message: types.Message, state: FSMContext):
    global m
    m = message
    await types.ChatActions.typing()
    
    await add_sender(message)

    photo  = message['photo'][-1]
    await photo.download(reference_images_path/photo['file_id'])
    image_url = await photo.get_url()
    dish = await async_search_image(url=image_url, env='prod')
    
    dish['photo_id'] = photo['file_id']

    sender = message['from'].to_python()
    dish['user_id'] = sender['id']
    dish['ml_version'] = ml_version 
    dish['timestamp']=pd.Timestamp.utcnow()


    async with state.proxy() as data: data['dish'] = dish.to_dict(orient = 'records')[0]

    dish.to_sql('dishes',schema = schema,if_exists = 'append',index = False,con=engine)
    
    await CState.photo_taken.set()
    async with state.proxy() as data: data['photo_id'] = photo['file_id']

    btns_text = tuple([(str(p)) for p in range(10,510,10)])

    keyboard_markup = types.ReplyKeyboardMarkup(row_width=2)
    keyboard_markup.add(*(types.KeyboardButton(text) for text in btns_text))


    await message.reply("set weight of the dish you are going to eat", reply_markup=keyboard_markup)


In [22]:
@dp.message_handler(lambda message: message.text.isdigit(), state=CState.photo_taken)
async def measure(message: types.Message, state: FSMContext):
    global dish

    grams = int(message.text)

    async with state.proxy() as data:  photo_id = data['photo_id']
    stmt = (
            dishes_table.update()
                        .where(dishes_table.c.photo_id == photo_id)
                        .values(grams=grams)
                        .returning(dishes_table.c.id)
    )
    session.execute(stmt)
    session.commit()

    async with state.proxy() as data:  dish = data['dish']

    energy = dish['energy']/100*grams

    today_consumed = pd.read_sql(f"""select energy,grams,timestamp from {schema}.dishes
                                 where user_id = {message['from']['id']} and timestamp > now() - interval '24 hours'
                                 and grams is not null;""",engine).set_index("timestamp")
    today_consumed= today_consumed['energy']/100*today_consumed['grams']
    user_tz = engine.execute(f"""select value from food.user_properties 
                                where user_id={message['from']['id']} and
                                property='tz'
                                order by id desc limit 1""").first()

    user_tz = user_tz[0] if user_tz else 'UTC'
    today_consumed = today_consumed.tz_convert(user_tz)
    now = pd.Timestamp.now(tz = user_tz)
    today_consumed = today_consumed.reset_index()	
    this_morning = pd.Timestamp(year = now.year,month = now.month,day = now.day,hour = 3,tz = user_tz)
    today_consumed = today_consumed[today_consumed['timestamp'] > pd.Timestamp(this_morning)][0].sum()
    today_consumed


    await message.reply(f"""you have consumed {energy} ccal with whis dish \n
                          today consumed {today_consumed}""")

In [23]:
if __name__ == '__main__':
    executor.start_polling(dp)

/home/dima/anaconda3/envs/food/lib/python3.9/site-packages/aiogram/types/mixins.py:40: DeprecationWarning: destination parameter is deprecated, please use destination_dir or destination_file.
  warn_deprecated(
/home/dima/food_main/food/search.py:48: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811803361/work/torch/csrc/utils/tensor_new.cpp:201.)
  series2tensor = lambda series:torch.tensor([np.array(c) for c in series.values])


0.19458224209473715
0.2133340731111528
0.21909561980704823
0.21958266716596989
0.21968082897707572
0.21968082897707567
0.21968082897707567


In [25]:
state = FSMContext()

TypeError: __init__() missing 3 required positional arguments: 'storage', 'chat', and 'user'

In [24]:
state = FSMContext??

Init signature: FSMContext(storage, chat, user)
Docstring:      <no docstring>
Source:        
class FSMContext:
    def __init__(self, storage, chat, user):
        self.storage: BaseStorage = storage
        self.chat, self.user = self.storage.check_address(chat=chat, user=user)

    def proxy(self):
        return FSMContextProxy(self)

    async def get_state(self, default: typing.Optional[str] = None) -> typing.Optional[str]:
        return await self.storage.get_state(chat=self.chat, user=self.user, default=default)

    async def get_data(self, default: typing.Optional[str] = None) -> typing.Dict:
        return await self.storage.get_data(chat=self.chat, user=self.user, default=default)

    async def update_data(self, data: typing.Dict = None, **kwargs):
        await self.storage.update_data(chat=self.chat, user=self.user, data=data, **kwargs)

    async def set_state(self, state: typing.Optional[typing.AnyStr] = None):
        await self.storage.set_state(chat=self.chat, use

In [22]:
dish

NameError: name 'dish' is not defined

In [42]:
m

<Message {"message_id":1837,"from":{"id":78882798,"is_bot":false,"first_name":"Dmitry","last_name":"Grankin","username":"dmitry_grankin","language_code":"en"},"chat":{"id":78882798,"first_name":"Dmitry","last_name":"Grankin","username":"dmitry_grankin","type":"private"},"date":1655215033,"photo":[{"file_id":"AgACAgIAAxkBAAIHLWKok7mf_Zhs_KkiyyGQnyPEOnQOAAK7vTEbrEpASSyndv5-F3HXAQADAgADcwADJAQ","file_unique_id":"AQADu70xG6xKQEl4","file_size":1231,"width":70,"height":90},{"file_id":"AgACAgIAAxkBAAIHLWKok7mf_Zhs_KkiyyGQnyPEOnQOAAK7vTEbrEpASSyndv5-F3HXAQADAgADbQADJAQ","file_unique_id":"AQADu70xG6xKQEly","file_size":12569,"width":248,"height":320},{"file_id":"AgACAgIAAxkBAAIHLWKok7mf_Zhs_KkiyyGQnyPEOnQOAAK7vTEbrEpASSyndv5-F3HXAQADAgADeAADJAQ","file_unique_id":"AQADu70xG6xKQEl9","file_size":52154,"width":619,"height":800},{"file_id":"AgACAgIAAxkBAAIHLWKok7mf_Zhs_KkiyyGQnyPEOnQOAAK7vTEbrEpASSyndv5-F3HXAQADAgADeQADJAQ","file_unique_id":"AQADu70xG6xKQEl-","file_size":77529,"width":991,"height":12

In [43]:
message = m

In [45]:
await add_sender(message)

photo  = message['photo'][-1]
await photo.download(reference_images_path/photo['file_id'])
image_url = await photo.get_url()
dish = await async_search_image(url=image_url, env='prod')


/home/dima/anaconda3/envs/food/lib/python3.9/site-packages/aiogram/types/mixins.py:40: DeprecationWarning: destination parameter is deprecated, please use destination_dir or destination_file.
  warn_deprecated(


0.19458224209473715
0.2133340731111528
0.21909561980704823
0.21958266716596989
0.21968082897707572
0.21968082897707567
0.21968082897707567


In [46]:

dish['photo_id'] = photo['file_id']


In [48]:
sender['id']

78882798

In [47]:

sender = message['from'].to_python()
dish['user_id'] = sender['id']
dish['ml_version'] = ml_version 
dish['timestamp']=pd.Timestamp.utcnow()
dish.to_sql('dishes',schema = schema,if_exists = 'append',index = False,con=engine)


TypeError: 'int' object is not subscriptable

In [ ]:

await CState.photo_taken.set()
async with state.proxy() as data: data['photo_id'] = photo['file_id']

btns_text = tuple([(str(p)) for p in range(10,510,10)])

keyboard_markup = types.ReplyKeyboardMarkup(row_width=2)
keyboard_markup.add(*(types.KeyboardButton(text) for text in btns_text))


await message.reply("set weight of the dish you are going to eat", reply_markup=keyboard_markup)


In [14]:
photo_id = 'AgACAgIAAxkBAAIG8WKoe7peZ5foW_cd-M-R9oTXbqRWAAKcvjEbmhxISQWeTbFDWlEcAQADAgADeQADJAQ'
grams = 20

In [ ]:
int(grams)

In [20]:
dishes_table.columns.keys()

['id',
 'description',
 'energy',
 'protein',
 'carb',
 'fat',
 'score',
 'photo_id',
 'user_id',
 'timestamp',
 'ml_version']

In [56]:
import urllib.parse
import tzwhere

In [87]:
geocode('lisboa')

(-75.094692, 4.5796933)

In [70]:
resp['properties']['lon']

84.881012

In [59]:
resp['lat'],resp['lon']

KeyError: 'lat'

In [ ]:
tz = tzwhere.tzwhere().tzNameAt(message.location.latitude, message.location.longitude)

In [16]:
import requests
from requests.structures import CaseInsensitiveDict

url = f"https://api.geoapify.com/v1/geocode/search?text=38%20Upper%20Montagu%20Street%2C%20Westminster%20W1H%201LJ%2C%20United%20Kingdom&apiKey=5d96ac126bcb462cb373297924ab2cb4"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"

resp = requests.get(url, headers=headers)

print(resp.status_code)

200


In [18]:
resp.json()

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license': 'Open Database License',
     'url': 'https://www.openstreetmap.org/copyright'},
    'name': 'Parkhaus Kö Talstraße',
    'housenumber': '1',
    'street': 'Talstraße',
    'suburb': 'Friedrichstadt',
    'district': 'Stadtbezirk 3',
    'city': 'Dusseldorf',
    'state': 'North Rhine-Westphalia',
    'postcode': '40217',
    'country': 'Germany',
    'country_code': 'de',
    'lon': 6.778318997521949,
    'lat': 51.21710155,
    'distance': 0,
    'result_type': 'amenity',
    'formatted': 'Parkhaus Kö Talstraße, Talstraße 1, 40217 Dusseldorf, Germany',
    'address_line1': 'Parkhaus Kö Talstraße',
    'address_line2': 'Talstraße 1, 40217 Dusseldorf, Germany',
    'category': 'parking.cars',
    'rank': {'popularity': 8.995467104553104},
    'place_id': '515fd9c0a7ff1c1b40599994ccfbc99b4940f0